### purpose 

calculate the fitness of populations in the climate outlier scenarios the multivariate sim

### notes

code inspired by 02_analysis/04_outlier_climate_runs/01_calculate_climate_outlier_fitness.ipynb

In [1]:
from pythonimports import *
from myfigs import histo_box

import MVP_00_start_pipeline as mvp00
import MVP_10_train_lfmm2_offset as mvp10
import MVP_summary_functions as mvp

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

t0 = dt.now()  # notebook timer

garden_dir = makedir('/home/b.lind/offsets/climate_outlier_runs/tutorial/garden_files')
fitness_dir = makedir(garden_dir.replace('garden_files', 'fitness_mats'))

lview, dview = get_client(cluster_id='1693573698-7b03', profile='lotterhos')

mvp.latest_commit()
session_info.show()

36 36
#########################################################
Today:	September 01, 2023 - 09:44:06 EDT
python version: 3.8.5
conda env: mvp_env

Current commit of pythonimports:
commit 9493334506a1f2926fba0734fa74ba2f87194611  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Fri Jun 30 09:41:00 2023 -0400

Current commit of MVP_offsets:
commit 59196546230c84b45ac4d8cd5c35052fdf910eb8  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Mon Aug 28 09:12:58 2023 -0400
#########################################################



# create climate data

convert std deviations to climate values for each climate variable

In [2]:
# standard deviations from climate center for temp and env2 (0 = climate center)
# standard deviations calculated in 02_analysis/04_outlier_climate_runs/05_visualize_validation_of_outlier_predictions.ipynb#stdev
deviations = [0, 1.72, 2.35, 2.74, 3.13, 3.53, 3.92, 4.31, 4.70, 5.09, 5.48, 5.88]

In [3]:
# get environmental data - I use this because deviations above were calculated from ...
    # pop-level data from 1- and 2-trait sims
gf_dir = '/home/b.lind/offsets/run_20220919_tutorial/gradient_forests/training/training_files'
envdata = pd.read_table(f'{gf_dir}/tutorial_envfile_GFready_pooled.txt', index_col=0)

envs = envdata.columns

envdata

,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
1,0.186225,-0.226935,0.861531,0.221974,-0.363974,0.761569
2,0.070353,-0.265649,0.799727,0.169512,-0.417965,0.315295
3,0.264633,-0.248372,0.375248,-0.172690,-0.586408,-0.320803
4,0.371616,0.167485,0.063118,-0.465978,-0.656053,-0.625240
5,0.244613,0.117437,-0.010939,-0.291486,-0.528748,-0.537376
...,...,...,...,...,...,...
96,-0.288220,0.261489,-0.317123,-0.018548,0.228030,-0.045559
97,-0.116570,0.323553,-0.231351,-0.269762,0.344912,0.059459
98,0.093915,0.400397,-0.150975,-0.408936,0.588880,0.274592
99,0.034177,0.399089,-0.440848,-0.464035,0.673533,0.360201


In [4]:
# fit standard scaler to landscape data, show example z scores from landscape data
scaler = StandardScaler()
scaler.fit(envdata[envs])

pd.DataFrame(scaler.transform(envdata[envs]),
             index=envdata.index,
             columns=envs)

,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
1,0.803091,-0.711353,3.107031,0.672132,-1.143354,2.475522
2,0.363314,-0.839008,2.888028,0.508052,-1.310047,1.024306
3,1.100677,-0.782039,1.383896,-0.562200,-1.830097,-1.044194
4,1.506714,0.589174,0.277869,-1.479470,-2.045116,-2.034178
5,1.024694,0.424150,0.015450,-0.933738,-1.652076,-1.748456
...,...,...,...,...,...,...
96,-0.997598,0.899135,-1.069507,-0.080113,0.684389,-0.149140
97,-0.346125,1.103779,-0.765574,-0.865796,1.045251,0.192364
98,0.452739,1.357160,-0.480766,-1.301070,1.798474,0.891945
99,0.226015,1.352846,-1.507925,-1.473394,2.059831,1.170333


In [5]:
# determine the environmental value given a standard deviation scenario
outlier_df = pd.DataFrame(columns=envs, index=deviations)
for z in deviations:
    edata = envdata.copy()
    for env in envs:
        edata[env] = z

    outlier_df.loc[z] = pd.DataFrame(
        scaler.inverse_transform(edata),
        index=None,
        columns=envs).loc[0]  # just first row since all vals per column are the same
    
outlier_df

,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
0.00,-0.025373,-0.011198,-0.015299,0.007067,0.006357,0.000304
1.72,0.427812,0.510438,0.470099,0.557019,0.563463,0.529233
2.35,0.593805,0.701503,0.64789,0.758455,0.767519,0.722968
2.74,0.696562,0.819781,0.757952,0.883154,0.893839,0.8429
3.13,0.799319,0.938059,0.868013,1.007852,1.020159,0.962832
3.53,0.904711,1.05937,0.980896,1.135748,1.149719,1.085838
3.92,1.007468,1.177647,1.090958,1.260446,1.276039,1.20577
4.31,1.110225,1.295925,1.201019,1.385145,1.40236,1.325701
4.70,1.212982,1.414203,1.31108,1.509843,1.52868,1.445633
5.09,1.31574,1.532481,1.421141,1.634541,1.655001,1.565565


In [6]:
for env, env_val in outlier_df.loc[0].items():
    print(env, env_val)

env1_mat -0.02537311851247319
env2_MTWetQ -0.011197646751946885
env3_MTDQ -0.015299279769150718
env4_PDM 0.007067339840601614
env5_PwarmQ 0.0063572111708173895
env6_PWM 0.00030400475412020733


In [7]:
# # determine the environmental value given a standard deviation scenario
# outlier_vals = defaultdict(dict)
# for env in envs:
#     print(ColorText(env).bold().green())
#     for dev in deviations:
#         outlier_vals[dev][env] = envdata[env].mean() + (dev * np.std(envdata[env]))
#         print(f'\t{dev} %s' % outlier_vals[dev][env])

In [8]:
# # make sure I can convert back to std dev from the val
# for (dev, env), val in unwrap_dictionary(outlier_vals):
#     newdev = (val - envdata[env].mean()) / envdata[env].std()
    
#     assert dev - np.round(newdev, 2) == 0

In [9]:
# outlier_df = pd.DataFrame(outlier_vals).T
# outlier_df

In [10]:
def create_new_envfiles():
    print(ColorText('\nCreating climate outlier environmental files').bold().custom('gold'))
    # read in arbitrary env file for correct formatting
    old_garden_dir = '/home/b.lind/offsets/run_20220919_tutorial/gradient_forests/fitting/garden_files'
    arbitrary_file = f'{old_garden_dir}/tutorial_envfile_GFready_pooled_1.txt'
    blank_env = pd.read_table(arbitrary_file, index_col=0)

    for col in blank_env.columns:
        assert luni(blank_env[col]) == 1

    new_envfiles = []
    for deviation in deviations:
        df = blank_env.copy()
        
        for env, env_val in outlier_df.loc[deviation].items():
            df[env] = env_val
            
        file = f'{garden_dir}/envfile_GF_ready_pooled_deviation_{deviation}.txt'
        
        df.to_csv(file, sep='\t', index=True)
        
        new_envfiles.append(file)

    return new_envfiles

In [11]:
envfiles = create_new_envfiles()

envfiles


Creating climate outlier environmental files


['/home/b.lind/offsets/climate_outlier_runs/tutorial/garden_files/envfile_GF_ready_pooled_deviation_0.txt',
 '/home/b.lind/offsets/climate_outlier_runs/tutorial/garden_files/envfile_GF_ready_pooled_deviation_1.72.txt',
 '/home/b.lind/offsets/climate_outlier_runs/tutorial/garden_files/envfile_GF_ready_pooled_deviation_2.35.txt',
 '/home/b.lind/offsets/climate_outlier_runs/tutorial/garden_files/envfile_GF_ready_pooled_deviation_2.74.txt',
 '/home/b.lind/offsets/climate_outlier_runs/tutorial/garden_files/envfile_GF_ready_pooled_deviation_3.13.txt',
 '/home/b.lind/offsets/climate_outlier_runs/tutorial/garden_files/envfile_GF_ready_pooled_deviation_3.53.txt',
 '/home/b.lind/offsets/climate_outlier_runs/tutorial/garden_files/envfile_GF_ready_pooled_deviation_3.92.txt',
 '/home/b.lind/offsets/climate_outlier_runs/tutorial/garden_files/envfile_GF_ready_pooled_deviation_4.31.txt',
 '/home/b.lind/offsets/climate_outlier_runs/tutorial/garden_files/envfile_GF_ready_pooled_deviation_4.7.txt',
 '/ho

In [12]:
for envfile in envfiles:
    print(envfile)
    display(pd.read_table(envfile, index_col=0))

/home/b.lind/offsets/climate_outlier_runs/tutorial/garden_files/envfile_GF_ready_pooled_deviation_0.txt


,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
1,-0.025373,-0.011198,-0.015299,0.007067,0.006357,0.000304
2,-0.025373,-0.011198,-0.015299,0.007067,0.006357,0.000304
3,-0.025373,-0.011198,-0.015299,0.007067,0.006357,0.000304
4,-0.025373,-0.011198,-0.015299,0.007067,0.006357,0.000304
5,-0.025373,-0.011198,-0.015299,0.007067,0.006357,0.000304
...,...,...,...,...,...,...
96,-0.025373,-0.011198,-0.015299,0.007067,0.006357,0.000304
97,-0.025373,-0.011198,-0.015299,0.007067,0.006357,0.000304
98,-0.025373,-0.011198,-0.015299,0.007067,0.006357,0.000304
99,-0.025373,-0.011198,-0.015299,0.007067,0.006357,0.000304


/home/b.lind/offsets/climate_outlier_runs/tutorial/garden_files/envfile_GF_ready_pooled_deviation_1.72.txt


,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
1,0.427812,0.510438,0.470099,0.557019,0.563463,0.529233
2,0.427812,0.510438,0.470099,0.557019,0.563463,0.529233
3,0.427812,0.510438,0.470099,0.557019,0.563463,0.529233
4,0.427812,0.510438,0.470099,0.557019,0.563463,0.529233
5,0.427812,0.510438,0.470099,0.557019,0.563463,0.529233
...,...,...,...,...,...,...
96,0.427812,0.510438,0.470099,0.557019,0.563463,0.529233
97,0.427812,0.510438,0.470099,0.557019,0.563463,0.529233
98,0.427812,0.510438,0.470099,0.557019,0.563463,0.529233
99,0.427812,0.510438,0.470099,0.557019,0.563463,0.529233


/home/b.lind/offsets/climate_outlier_runs/tutorial/garden_files/envfile_GF_ready_pooled_deviation_2.35.txt


...

,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
1,1.31574,1.532481,1.421141,1.634541,1.655001,1.565565
2,1.31574,1.532481,1.421141,1.634541,1.655001,1.565565
3,1.31574,1.532481,1.421141,1.634541,1.655001,1.565565
4,1.31574,1.532481,1.421141,1.634541,1.655001,1.565565
5,1.31574,1.532481,1.421141,1.634541,1.655001,1.565565
...,...,...,...,...,...,...
96,1.31574,1.532481,1.421141,1.634541,1.655001,1.565565
97,1.31574,1.532481,1.421141,1.634541,1.655001,1.565565
98,1.31574,1.532481,1.421141,1.634541,1.655001,1.565565
99,1.31574,1.532481,1.421141,1.634541,1.655001,1.565565


/home/b.lind/offsets/climate_outlier_runs/tutorial/garden_files/envfile_GF_ready_pooled_deviation_5.48.txt


,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
1,1.418497,1.650759,1.531203,1.75924,1.781321,1.685496
2,1.418497,1.650759,1.531203,1.75924,1.781321,1.685496
3,1.418497,1.650759,1.531203,1.75924,1.781321,1.685496
4,1.418497,1.650759,1.531203,1.75924,1.781321,1.685496
5,1.418497,1.650759,1.531203,1.75924,1.781321,1.685496
...,...,...,...,...,...,...
96,1.418497,1.650759,1.531203,1.75924,1.781321,1.685496
97,1.418497,1.650759,1.531203,1.75924,1.781321,1.685496
98,1.418497,1.650759,1.531203,1.75924,1.781321,1.685496
99,1.418497,1.650759,1.531203,1.75924,1.781321,1.685496


/home/b.lind/offsets/climate_outlier_runs/tutorial/garden_files/envfile_GF_ready_pooled_deviation_5.88.txt


,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
1,1.523889,1.77207,1.644086,1.887135,1.91088,1.808503
2,1.523889,1.77207,1.644086,1.887135,1.91088,1.808503
3,1.523889,1.77207,1.644086,1.887135,1.91088,1.808503
4,1.523889,1.77207,1.644086,1.887135,1.91088,1.808503
5,1.523889,1.77207,1.644086,1.887135,1.91088,1.808503
...,...,...,...,...,...,...
96,1.523889,1.77207,1.644086,1.887135,1.91088,1.808503
97,1.523889,1.77207,1.644086,1.887135,1.91088,1.808503
98,1.523889,1.77207,1.644086,1.887135,1.91088,1.808503
99,1.523889,1.77207,1.644086,1.887135,1.91088,1.808503


# calc fitness to each scenario

In [13]:
def calc_R_fitness(*args):
    # also used in 02.05.01_estimate_fitness.ipynb
    import subprocess
    
    args = [str(arg) for arg in args]
    
    Rscript = '/home/b.lind/anaconda3/envs/MVP_env_R4.0.3/bin/Rscript'

    script_file = '/home/b.lind/code/MVP-offsets/01_src/MVP_climate_outlier_fitness_calculator.R'
    
    output = subprocess.check_output(
        [
            Rscript,
            script_file,
            *args
        ]
    )

    return output.decode('utf-8')

In [14]:
outlier_df

,env1_mat,env2_MTWetQ,env3_MTDQ,env4_PDM,env5_PwarmQ,env6_PWM
0.00,-0.025373,-0.011198,-0.015299,0.007067,0.006357,0.000304
1.72,0.427812,0.510438,0.470099,0.557019,0.563463,0.529233
2.35,0.593805,0.701503,0.64789,0.758455,0.767519,0.722968
2.74,0.696562,0.819781,0.757952,0.883154,0.893839,0.8429
3.13,0.799319,0.938059,0.868013,1.007852,1.020159,0.962832
3.53,0.904711,1.05937,0.980896,1.135748,1.149719,1.085838
3.92,1.007468,1.177647,1.090958,1.260446,1.276039,1.20577
4.31,1.110225,1.295925,1.201019,1.385145,1.40236,1.325701
4.70,1.212982,1.414203,1.31108,1.509843,1.52868,1.445633
5.09,1.31574,1.532481,1.421141,1.634541,1.655001,1.565565


In [15]:
fitness_dir

'/home/b.lind/offsets/climate_outlier_runs/tutorial/fitness_mats'

In [16]:
jobs = []
args = []
for deviation, (climate_optima) in outlier_df.iterrows():
    outfile = op.join(fitness_dir, f'tutorial_{deviation}.txt')
    
    arg = [
        'tutorial',
        outfile,
        *climate_optima
    ]
    
    args.append(arg)
    
    jobs.append(
        lview.apply_async(
            calc_R_fitness, *(arg)
        )
    )
    
watch_async(jobs)


Watching 12 jobs ...


100%|███████████████| 12/12 [00:00<00:00, 28.36it/s]


In [17]:
fitness_matfiles = fs(fitness_dir, endswith='.txt')

len(fitness_matfiles)

12

In [18]:
for f in fitness_matfiles:
    df = pd.read_table(f)
    
    print(op.basename(f), df.shape)
    display(df.head())

tutorial_0.0.txt (1, 100)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
1,0.738889,0.786154,0.79,0.737,0.81,0.930833,0.762222,0.772222,0.862,0.921,0.81,0.852222,0.81,0.73625,0.823333,0.886667,0.818182,0.834,0.888182,0.937273,0.9275,0.8675,0.738571,0.683077,0.845833,0.88,0.77125,0.86,0.953077,0.926364,0.86125,0.825,0.755,0.88,0.930909,0.825714,0.760833,0.90625,0.918,0.941111,0.7975,0.797,0.815556,0.9275,0.9275,0.868571,0.826,0.914667,0.876,0.915714,0.77625,0.798,0.883636,0.944615,0.942727,0.93125,0.888571,0.874286,0.853846,0.882222,0.848462,0.85,0.89375,0.898182,0.92125,0.927895,0.851,0.81125,0.87,0.8075,0.8775,0.9,0.880667,0.87,0.93,0.932727,0.855455,0.848571,0.828889,0.79,0.887143,0.902,0.842,0.879231,0.937,0.924545,0.87,0.853333,0.774667,0.73,0.906667,0.883158,0.843333,0.9,0.927778,0.909,0.897143,0.836154,0.75,0.734615


tutorial_1.72.txt (1, 100)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
1,0.681111,0.622308,0.46,0.356,0.447143,0.581667,0.731111,0.672222,0.603333,0.668,0.584286,0.5,0.37625,0.38125,0.462222,0.684167,0.720909,0.627,0.523636,0.62,0.4925,0.436667,0.382857,0.412308,0.5475,0.709167,0.715,0.576667,0.575385,0.563636,0.38375,0.402857,0.421,0.486667,0.626364,0.741429,0.638333,0.60125,0.563,0.597778,0.37625,0.46,0.514444,0.5575,0.649167,0.714286,0.639,0.599333,0.53,0.58,0.41125,0.492,0.642727,0.65,0.700909,0.6825,0.612857,0.548571,0.52,0.58,0.477692,0.582857,0.78375,0.732727,0.67,0.616316,0.519,0.475,0.59875,0.56,0.51875,0.665333,0.785333,0.696667,0.62125,0.588182,0.515455,0.537143,0.583889,0.52,0.56,0.712,0.752,0.647692,0.624,0.59,0.52,0.607778,0.589333,0.51,0.614444,0.758947,0.726667,0.67,0.64,0.603,0.614286,0.678462,0.571429,0.523846


tutorial_2.35.txt (1, 100)


...

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
1,0.055556,0.04,0.02,0.01,0.011429,0.0225,0.068889,0.051111,0.030667,0.035,0.034286,0.016667,0.01,0.01,0.015556,0.04,0.058182,0.037,0.017273,0.028182,0.01,0.01,0.012857,0.013077,0.024167,0.045,0.06375,0.024444,0.020769,0.020909,0.01,0.01,0.011,0.012222,0.03,0.058571,0.048333,0.02625,0.02,0.024444,0.01,0.016,0.02,0.0225,0.031667,0.047143,0.04,0.026,0.02,0.022857,0.01,0.019,0.035455,0.030769,0.038182,0.035,0.03,0.024286,0.02,0.024444,0.014615,0.025714,0.06125,0.049091,0.0325,0.027895,0.023,0.01625,0.03,0.025,0.0175,0.036,0.062,0.043333,0.02625,0.022727,0.019091,0.022857,0.028889,0.02,0.024286,0.042,0.06,0.033846,0.029,0.022727,0.018,0.032222,0.032667,0.024,0.028889,0.056316,0.053333,0.035556,0.03,0.027,0.03,0.045385,0.032857,0.026923


tutorial_5.48.txt (1, 100)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
1,0.034444,0.023077,0.01,0.003,0.008571,0.011667,0.043333,0.031111,0.016,0.019,0.02,0.011111,0.00125,0.00875,0.008889,0.025833,0.033636,0.022,0.01,0.016364,0.01,0.005833,0.002857,0.01,0.0125,0.0275,0.03875,0.016667,0.010769,0.010909,0,0.003571,0.009,0.01,0.017273,0.038571,0.0275,0.0125,0.01,0.012222,0.0025,0.01,0.011111,0.0125,0.019167,0.031429,0.025,0.013333,0.01,0.012857,0.01,0.01,0.019091,0.02,0.022727,0.02375,0.018571,0.014286,0.010769,0.014444,0.009231,0.012857,0.0375,0.03,0.02,0.014737,0.013,0.01,0.01875,0.015,0.01,0.020667,0.038,0.025556,0.01625,0.011818,0.01,0.012857,0.017778,0.01,0.014286,0.028,0.039,0.02,0.015,0.012727,0.01,0.02,0.017333,0.012,0.016667,0.033684,0.031667,0.021111,0.016667,0.014,0.017143,0.025385,0.02,0.016923


tutorial_5.88.txt (1, 100)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
1,0.021111,0.013846,0.006,0,0.001429,0.01,0.023333,0.016667,0.01,0.01,0.012857,0.005556,0,0,0.002222,0.011667,0.020909,0.013,0.006364,0.01,0,0,0.001429,0.003077,0.008333,0.015833,0.02375,0.007778,0.009231,0.01,0,0,0.001,0.001111,0.009091,0.022857,0.0175,0.01,0.01,0.01,0,0.003,0.007778,0.005,0.010833,0.014286,0.011,0.01,0.009,0.01,0,0.007,0.01,0.010769,0.011818,0.01,0.008571,0.005714,0.008462,0.01,0.002308,0.01,0.01875,0.016364,0.01,0.01,0.005,0.00375,0.01,0.01,0.00625,0.012,0.023333,0.015556,0.01,0.01,0.009091,0.008571,0.01,0.01,0.005714,0.012,0.02,0.01,0.01,0.01,0.007,0.011111,0.010667,0.01,0.01,0.018947,0.016667,0.01,0.01,0.01,0.01,0.015385,0.01,0.01


In [19]:
# make sure no errors
for j in jobs:
    x = j.r  # returns None if no error

In [20]:
formatclock(dt.now() - t0)

'0-00:00:03'